第一步：安装和导入必要包

In [ ]:
!pip install datasets
!pip install evaluate

In [ ]:
from datasets import load_dataset
from transformers import AutoTokenizer, DataCollatorWithPadding, AutoModelForSequenceClassification, import TrainingArguments, Trainer
import numpy as np
import evaluate

第二步：原始数据加载

In [ ]:
dataset = load_dataset("glue", "mrpc")

第三步：数据处理

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("google-bert/bert-base-cased")


def tokenize_function(examples):
    return tokenizer(examples["sentence1"], examples["sentence2"], padding="max_length", truncation=True, max_length=128)


tokenized_datasets = dataset.map(tokenize_function, batched=True)
data_collator = DataCollatorWithPadding(tokenizer)

第四步：构建模型

In [ ]:
model = AutoModelForSequenceClassification.from_pretrained("google-bert/bert-base-cased", num_labels=2)
metric = evaluate.load("accuracy")


def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)


training_args = TrainingArguments(output_dir="test_trainer",
                  eval_strategy="epoch",
                  per_device_train_batch_size=8,
                  per_device_eval_batch_size=8,
                  num_train_epochs=5,
                  logging_steps=10,
                  )
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets['train'],
    eval_dataset=tokenized_datasets['validation'],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

trainer.train()

第五步：模型保存与加载

In [ ]:
trainer.save_model('./my_model/')

In [ ]:
model = AutoModelForSequenceClassification.from_pretrained("./my_model/")

第六步：模型推理

In [ ]:
from transformers import pipeline

# 加载文本分类pipeline
classifier = pipeline("text-classification", model="./my_model/", tokenizer="google-bert/bert-base-cased", device=0)

# 输入文本进行推理
texts = {"text": "this is a good thing for me", "text_pair": "it is a good news"}
results = classifier(texts)

# 输出结果
print(results)
# for result in results:
#     print(f"Label: {result['label']}, Score: {result['score']:.4f}")